Привет, меня зовут Артем. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Над этим проектом нужно будет еще немного поработать. Однако, изменения не должны занять много времени.
* В работе я оставил несколько советов. Буду рад, если ты учтешь их.
* С радостью отвечу на твои вопросы, если они есть. Лучше всего их собрать в следующей ячейке. Жду новую версию проекта.

### <font color='orange'>Общее впечатление (ревью 2)</font>
* После исправлений проект улучшился и теперь он может быть зачтен.
* Для удобства все новые комментарии обозначены фразой "ревью 2".
* Удачи в дальнейшем обучении и следующих работах!

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from scipy.stats import bootstrap
from scipy import stats as st
import numpy as np
from math import sqrt

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [3]:
try:
    geo0 = pd.read_csv('geo_data_0.csv')
    geo1 = pd.read_csv('geo_data_1.csv')
    geo2 = pd.read_csv('geo_data_2.csv')
except:
    geo0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    geo1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    geo2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

geo0.index = geo0.id
geo0.drop(['id'], axis=1, inplace=True)
geo1.index = geo1.id
geo1.drop(['id'], axis=1, inplace=True)
geo2.index = geo2.id
geo2.drop(['id'], axis=1, inplace=True)

In [4]:
# Проверим данные
display(geo0.info())
display(geo1.info())
display(geo2.info())
display(geo0.describe())
display(geo1.describe())
display(geo2.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Пропущенных значений нет, у датасета geo2 в первых двух столбцах большое стандартное отклонение. Минимальные и максимальные значения тоже сильно отличаются между столбцами, поэтому понадобится нормализация.

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены и осмотрены корректно. Согласен, что колонка id нам не нужна.
</div>

## Обучение и проверка модели

In [5]:
# Подготовим данные
x0, y0 = geo0[['f0', 'f1', 'f2']], geo0['product']
x1, y1 = geo1[['f0', 'f1', 'f2']], geo1['product']
x2, y2 = geo2[['f0', 'f1', 'f2']], geo2['product']

X_train0, X_test0, y_train0, y_test0 = train_test_split(x0, y0, train_size=0.75, random_state=44)
X_train1, X_test1, y_train1, y_test1 = train_test_split(x1, y1, train_size=0.75, random_state=44)
X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, train_size=0.75, random_state=44)

In [6]:
 # Создадим модели и обучим модели
pipe = Pipeline([('scaler', StandardScaler()), ('LinearRegression', LinearRegression())])
model0 = pipe.fit(X_train0, y_train0)
pipe = Pipeline([('scaler', StandardScaler()), ('LinearRegression', LinearRegression())])
model1 = pipe.fit(X_train1, y_train1)
pipe = Pipeline([('scaler', StandardScaler()), ('LinearRegression', LinearRegression())])
model2 = pipe.fit(X_train2, y_train2)

print('model0 score', model0.score(X_test0, y_test0))
print('model1 score', model1.score(X_test1, y_test1))
print('model2 score', model2.score(X_test2, y_test2))

model0 score 0.2660329817014818
model1 score 0.9996298568202542
model2 score 0.19487075168240908


In [7]:
# Попробуем оценить качество моделей при помощи кросс-валидации
pipe = Pipeline([('scaler', StandardScaler()), ('LinearRegression', LinearRegression())])
model_cv0 = cross_val_score(pipe, x0, y0, cv=5)
model_cv1 = cross_val_score(pipe, x1, y1, cv=5)
model_cv2 = cross_val_score(pipe, x2, y2, cv=5)

cv_res = pd.concat([pd.Series(model_cv0), pd.Series(model_cv1), pd.Series(model_cv2)], axis=1)

print(cv_res := pd.concat(
    [pd.Series(cv_res.mean(), name='mean'), pd.Series(cv_res.std(), name='sigma')],
    axis=1
))

       mean     sigma
0  0.275491  0.006089
1  0.999624  0.000005
2  0.198716  0.004868


In [8]:
# Сделаем предсказания
prediction0 = model0.predict(X_test0)
prediction1 = model1.predict(X_test1)
prediction2 = model2.predict(X_test2)

print('model 0 prediction mean', prediction0.mean())
print('model 1 prediction mean', prediction1.mean())
print('model 2 prediction mean', prediction2.mean())

print('RMSE', sqrt(mse0 := mean_squared_error(y_test0, prediction0)))
print('RMSE', sqrt(mse1 := mean_squared_error(y_test1, prediction1)))
print('RMSE', sqrt(mse2 := mean_squared_error(y_test2, prediction2)))

model 0 prediction mean 92.46203438661259
model 1 prediction mean 68.51920639719152
model 2 prediction mean 94.98365106986161
RMSE 37.77268636795981
RMSE 0.8857076441957261
RMSE 40.10562308381635


В датасетах мало признаков, поэтому модели получились довольно низкого качества, кроме модели для датасета 1. Предсказанная средняя почти одинаковая у всех моделей, но ошибка меньше всего первой модели.

<div class="alert alert-block alert-success">
<b>Успех:</b> Модели обучены корректно.
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> На протяжении всего проекта происходит работа с 3мя одинаковыми таблицами. В связи с этим можно было использовать функции и/или циклы, чтобы избежать копирования кода.
<br> Зачем его нужно избегать? Потому что очень часто оно приводит к ошибкам (например, забыл поменять одну из переменных). Впоследствии такие ошибки очень трудно найти.
</div>

## Подготовка к расчёту прибыли

In [9]:
# Подготовка констант для расчёта
BOREHOLE_AMOUNT = 500
TOP_BOREHOLE_IN_DEVELOPMENT_AMOUNT = 200
DEVELOPMENT_BUDGET = 10_000_000_000
UNIT_OF_PRODUCT = 450_000
# Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
MIN_UNITS_IN_BOREHOLE = DEVELOPMENT_BUDGET/TOP_BOREHOLE_IN_DEVELOPMENT_AMOUNT/UNIT_OF_PRODUCT
print('Минимальное безубыточное количество единиц нефти в скважине', MIN_UNITS_IN_BOREHOLE)
print('Предсказанная средняя минус безубыточное количество единиц в скважине',
      pred0_diff_with_optimum := prediction0.mean() - MIN_UNITS_IN_BOREHOLE)
print('Предсказанная средняя минус безубыточное количество единиц в скважине',
      pred1_diff_with_optimum := prediction1.mean() - MIN_UNITS_IN_BOREHOLE)
print('Предсказанная средняя минус безубыточное количество единиц в скважине',
      pred2_diff_with_optimum := prediction2.mean() - MIN_UNITS_IN_BOREHOLE)

Минимальное безубыточное количество единиц нефти в скважине 111.11111111111111
Предсказанная средняя минус безубыточное количество единиц в скважине -18.649076724498528
Предсказанная средняя минус безубыточное количество единиц в скважине -42.59190471391959
Предсказанная средняя минус безубыточное количество единиц в скважине -16.127460041249506


В среднем во всех регионах не хватает 16 тысяч баррелей. Всего для безубыточной добычи из скважины она должна содержать не менее 111.12 тысяч баррелей.

<div class="alert alert-block alert-success">
<b>Успех:</b> Точка безубыточности найдена верно, сравнение проведено!
</div>

## Расчёт прибыли и рисков 


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Этот шаг нужно будет значительно переработать.
    
1. В предыдущих шагах было получено 25000 настоящих и предсказанных значений для каждого региона.
2. Здесь нужно определить функцию, которая на вход будет принимать 500 настоящих и 500 предсказанных значений для каждого региона. Для выбора 200 лучших скважин нужно использовать предсказанные значения. А вот для подсчета прибыли должны использоваться настоящие значения соответствующих ячеек.
3. В следующем шаге нужно семплить 500 случайных скважин и передавать их в функцию для подсчета прибыли.
4. В результате у тебя получится 1000 значений прибыли для каждого региона. По нему нужно посчитать 3 статистики:
    * Среднюю прибыль
    * Интервал с помощью метода квантилей
    * Риск убытков – долю случаев, в которых прибыль отрицательна.
</div>

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Сделал ниже
</div>

In [10]:
state = np.random.RandomState(44)

# Добавляем индекс в предсказание
def add_index(predict, y_test):
    return pd.Series(predict, index=y_test.index)

# Нахождение лучших скважин и расчёт прибыли
def bh_select_and_profit(pred, test):
    sorted_ = pred.sort_values(ascending=False)[:200]
    return test[sorted_.index].sum()*UNIT_OF_PRODUCT-DEVELOPMENT_BUDGET

# Выборка 500 скважин
def sample_500(predict, y_test):
    predicted_values = predict.sample(500, random_state=state)
    test_values = y_test.loc[predicted_values.index]
    return predicted_values, test_values

# Соберём всё в кортежи
predictions = (prediction0, prediction1, prediction2)
test_values = (y_test0, y_test1, y_test2)

# Добавляем индекс в предсказание
predict_with_index = [add_index(pred, ind) for pred, ind in zip(predictions, test_values)]

# Соберём результаты в списки по регионам
results = {'region0': [], 'region1': [], 'region2': []}
for predict, test_val, region in zip(predict_with_index, test_values, results):
    results[region] = [bh_select_and_profit(*sample_500(predict, test_val)) for i in range(1000)]

# Расчитаем искомые показатели
results_table = pd.DataFrame(results)
# Средняя прибыль
print('Средняя прибыль')
print(results_table.mean())
# Доверительный интервал
region0_low, region0_high = results_table.region0.quantile(0.025), results_table.region0.quantile(0.975)
region1_low, region1_high = results_table.region1.quantile(0.025), results_table.region1.quantile(0.975)
region2_low, region2_high = results_table.region2.quantile(0.025), results_table.region2.quantile(0.975)
print(f'Доверительный интервал 0: {region0_low:.3e}, {region0_high:.3e}')
print(f'Доверительный интервал 1: {region1_low:.3e}, {region1_high:.3e}')
print(f'Доверительный интервал 2: {region2_low:.3e}, {region2_high:.3e}')
# Риск убытков
print('Риск убытков')
print(results_table.apply(lambda x: sum(x<0)/len(x), axis=0))

Средняя прибыль
region0    3.792515e+08
region1    4.489684e+08
region2    3.257170e+08
dtype: float64
Доверительный интервал 0: -1.257e+08, 8.986e+08
Доверительный интервал 1: 6.525e+07, 8.467e+08
Доверительный интервал 2: -1.553e+08, 8.297e+08
Риск убытков
region0    0.067
region1    0.011
region2    0.103
dtype: float64


__Вывод__: Выгоден для добычи только регион номер 1. С вероятностью 95% он окажется прибыльным. Также у него наибольшая средняя прибыль из всех и минимальный риск убытков.

Теперь мне хотелось бы задать 2 вопроса. Вообще в целом смысл задачи не очень понятен. Есть данные, в них уже есть цифры, почему не выбрать скважины по ним просто, для чего в этом случае предсказание? Я понимаю если бы были какие-то неразмеченные данные, но здесь то всё есть. Ниже ошибка которую я не смог исправить, я там ответил на ваш комментарий.

In [ ]:
# Используем метод bootstrap для оценки доверительного интервала
state = np.random.RandomState(44)
bootstrap(data=(list(prediction0), ), 
          statistic=lambda x, axis: sum(x) * UNIT_OF_PRODUCT - DEVELOPMENT_BUDGET,
          n_resamples=1000,
          batch=200,
          vectorized=True,
          confidence_level=0.95,
          axis=0,
          random_state=state)

<div class="alert alert-block alert-info">
<b>Совет (ревью 2): </b> Вот так запускается код:
    
1) в data должно быть обязательно 2 размерности. Это есть в примере в доке: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html

2) sum() уже нужно применять к list.
</div>

In [38]:
# Используем метод bootstrap для оценки доверительного интервала
state = np.random.RandomState(44)
bootstrap(data=(list(prediction0), ), 
          statistic=lambda x, axis: sum(x) * UNIT_OF_PRODUCT - DEVELOPMENT_BUDGET,
          n_resamples=1000,
          batch=200,
          vectorized=True,
          confidence_level=0.95,
          axis=0,
          random_state=state)

/opt/conda/lib/python3.9/site-packages/scipy/stats/_resampling.py:124: RuntimeWarning: invalid value encountered in double_scalars
  alpha_1 = ndtr(z0_hat + num1/(1 - a_hat*num1))
/opt/conda/lib/python3.9/site-packages/scipy/stats/_resampling.py:126: RuntimeWarning: invalid value encountered in double_scalars
  alpha_2 = ndtr(z0_hat + num2/(1 - a_hat*num2))
/opt/conda/lib/python3.9/site-packages/scipy/stats/_resampling.py:92: DegenerateDataWarning: The bootstrap distribution is degenerate; the confidence interval is not defined.
  warnings.warn(DegenerateDataWarning(msg))


BootstrapResult(confidence_interval=ConfidenceInterval(low=nan, high=nan), standard_error=149347797.18917978)

__Вопрос__ уважаемому ревьюеру, почему у меня вылезает такая ошибка?

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Параметр axis должен быть типа int. 
</div>

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Я проверял, ставить int не помогает, полностью убрать параметр тоже не помогает. Изначально запускал вообще без axis, так как на вход pd.Series.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован